### Data cleaner project for Clean Coalition
```
tyler@clean-coalition.org
Tyler Lewis
February 2023
```


In [4]:
read_filename = './AACI_Health_Center_raw_data.csv'
write_filename = './data_cleaned.csv'

In [5]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
%matplotlib inline
import csv
import pandas as pd
from datetime import datetime
from datetime import timedelta

# 1 week of 15 min interval
entries_per_week = 672 

# open dataframe using @pandas
df = pd.read_csv(read_filename) 

In [6]:
def to_datetime(date_string):
    """
    Converts a string to a datetime object.

    Parameters:
        date_string (str): A string representing a date and time in UtilityAPI format.

    Returns:
        datetime: A datetime object representing the input date and time.
    """
    date_obj = datetime.strptime(date_string, '%m/%d/%Y %H:%M')
    return date_obj

In [24]:
# df = pd.read_csv(read_filename, index_col='interval_start', parse_dates=True)

# # Clean up the data if necessary
# df = df.dropna() # Remove any missing values

# # Resample the data into daily intervals
# daily_data = df.resample('D').sum()

# # Create a line plot of the daily data
# fig, ax = plt.subplots()
# ax.plot(daily_data.index, daily_data['net_kWh'])
# ax.set_xlabel('Time')
# ax.set_ylabel('Net Energy Usage (kWh)')
# ax.set_title('Residential Energy Usage - Daily')

# # Modify the format of the x-axis labels
# date_form = DateFormatter('%m-%d')
# ax.xaxis.set_major_formatter(date_form)

In [22]:
plt.show()

The code here simply needs to 
- fetch data from exactly 1 week prior to the lapse in data, for 5 cycles (75 minutes, 4500 seconds)
- write 5 rows in every 'else' instance

In [21]:
def finddirtyrows(df):
    values=[]
    for ind in df.index:
        # Store variables for current index of dataframe
        interval_start = to_datetime(df['interval_start'][ind])
        interval_end = to_datetime(df['interval_end'][ind])
        _timedelta = interval_end - interval_start
        
        if (_timedelta != timedelta(minutes = 15)):
            # improper entry
            values.append(ind)

    print(values)

In [22]:
finddirtyrows(df)

[28304, 63240, 98848]


In [23]:
def writecleandata(csvwriter, df):
    # a={}
    for ind in df.index:
        # Store variables for current index of dataframe
        interval_start = to_datetime(df['interval_start'][ind])
        interval_end = to_datetime(df['interval_end'][ind])
        _timedelta = interval_end - interval_start
        
        if (_timedelta == timedelta(minutes = 15)):
            # proper entry, no modification need
            csvwriter.writerow(df.iloc[ind])
        else:
            # There is data missing in this set at the current index
            # Will copy usage data from week prior for most-accurate fix
            
            copy_index = ind - entries_per_week
            ph_interval_end = interval_end
            
            while (ph_interval_end - interval_start >= timedelta(minutes = 15)):
                copy_curr = df.iloc[copy_index].copy()
                copy_curr.interval_end = ph_interval_end
                copy_curr.interval_start = ph_interval_end - timedelta(minutes = 15)
                
                csvwriter.writerow(copy_curr)
                
                ph_interval_end = ph_interval_end - timedelta(minutes = 15)

    print('complete')
    

#### In case of incorrect entry:
1. Create placeholder interval_end object, set to current interval_end
2. Insert record into csv with interval_start = interval_end - 15 minutes
3. set interval_end to interval_start and repeat step 2
4. loop steps 2/3 until interval_start = original_interval_start

In [14]:
def start():
    with open(write_filename, 'w', newline='') as csvfile: 

        # creating a csv writer object @csv
        csvwriter = csv.writer(csvfile) 

        # writing the column headers for write file @csv
        csvwriter.writerow(df.columns.values) 

        # writing the data rows 
        writecleandata(csvwriter, df)

In [25]:
start()

complete


# End 

print(df.columns.values)

In [8]:
obj = df.iloc[0]
obj.utility = 'hi'
print(obj)

meter_uid                                               1201327
utility                                                      hi
utility_service_id                                   2887177833
utility_service_address    2400 MOORPARK AVE SAN JOSE, CA 95128
utility_meter_number                                 1010786945
utility_tariff_name                                       HB19S
interval_start                                 01/01/2023 23:45
interval_end                                   01/02/2023 00:00
interval_kWh                                               18.4
fwd_kWh                                                     NaN
max_kW                                                     73.6
net_kWh                                                    18.4
rev_kWh                                                     NaN
source                                                      NaN
updated                                                     NaN
interval_timezone                       

C:\Users\tyler\AppData\Local\Temp\ipykernel_20704\677083167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obj.utility = 'hi'


In [21]:
ind = 0
_timedelta = to_datetime(df['interval_end'][ind]) - to_datetime(df['interval_start'][ind])
print(_timedelta/timedelta(minutes = 15))
# for i in range(_timedelta/timedelta(minutes = 15)):
#                 print(i)

1.0
